In [ ]:
import os
import numpy as np
np.random.seed(69)
import pandas as pd
import random
import pickle as pkl
import matplotlib.pyplot as plt
import matplotlib.image as img
import seaborn as sns
import tensorflow as tf
from tqdm.notebook import tqdm
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, ZeroPadding2D, LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import load_model
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# import kerastuner as kt
# from kerastuner import HyperModel
import time

In [ ]:
df = pd.read_csv("../input/sdss-project-v10/Reg_SDSS_Query_v1.0_DF.csv",index_col=0)

In [ ]:
photodf = df.loc[:,['dered_u', 'deVRad_u', 'psffwhm_u', 'extinction_u',
       'dered_g', 'deVRad_g', 'psffwhm_g', 'extinction_g', 'dered_r',
       'deVRad_r', 'psffwhm_r', 'extinction_r', 'dered_i', 'deVRad_i',
       'psffwhm_i', 'extinction_i', 'dered_z', 'deVRad_z', 'psffwhm_z',
       'extinction_z', 'u_g', 'g_r', 'r_i', 'i_z']]

y_spec = df["redshift"].values
X = photodf.values

In [ ]:
X_train, X_test, y_spec_train, y_spec_test = train_test_split(X, y_spec, test_size=0.2, random_state=69)

In [ ]:
model = Sequential()
model.add(Dense(1024, activation="sigmoid", input_dim=X_train.shape[1]))
model.add(Dropout(0.25))
model.add(Dense(512, activation="sigmoid"))
model.add(Dropout(0.25))
model.add(Dense(256, activation="sigmoid"))
model.add(Dropout(0.25))
model.add(Dense(128, activation="sigmoid"))
model.add(Dropout(0.25))
model.add(Dense(64, activation="sigmoid"))
model.add(Dropout(0.25))
model.add(Dense(32, activation="sigmoid"))
model.add(Dropout(0.25))
model.add(Dense(1, activation="relu"))

model.compile(loss="mean_squared_error",optimizer="adam")

In [ ]:
es = EarlyStopping(monitor='val_loss', verbose=0, patience=100, restore_best_weights=True)
cb = [es]

model.fit(X_train, y_spec_train, validation_data = (X_test, y_spec_test),verbose=2, epochs=2000,batch_size=2048,callbacks=cb)

In [ ]:
model.save("DNNRegressor.h5")

In [ ]:
df2 = pd.read_csv("../input/sdss-project-v10/SDSS_Query_v1.0_DF.csv",index_col=0)

photodf2 = df2.loc[:,['dered_u', 'deVRad_u', 'psffwhm_u', 'extinction_u',
       'dered_g', 'deVRad_g', 'psffwhm_g', 'extinction_g', 'dered_r',
       'deVRad_r', 'psffwhm_r', 'extinction_r', 'dered_i', 'deVRad_i',
       'psffwhm_i', 'extinction_i', 'dered_z', 'deVRad_z', 'psffwhm_z',
       'extinction_z', 'u_g', 'g_r', 'r_i', 'i_z']]

y_spec2 = df2["redshift"].values
X2 = photodf2.values

y_pred2 = model.predict(X2)

print(f"MSE Error on Test is {metrics.mean_squared_error(y_spec2,y_pred2)}")
print(f"MAE Error on Test is {metrics.mean_absolute_error(y_spec2,y_pred2)}")
print(f"R2 Score on Test is {metrics.r2_score(y_spec2,y_pred2)}")